<a href="https://colab.research.google.com/github/mrospond/kik/blob/main/W06_kody_liniowe_wst%C4%99p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Potrzebne biblioteki...

# Praca z wektorami i macierzami
import numpy as np

# Kodowanie kanałowe: kody liniowe

Interesują nas tylko tzw. **blokowe** kody **liniowe** (czyli opisywane macierzami). Kodowanie od strony algebraicznej dla kodu o parametrach:
$$ (n,k) $$
polega na utworzeniu słowa kodowego o długości
$$ n \text{ symboli} $$
na podstawie ciągu informacyjnego liczącego
$$ k \text{ symboli} $$
Kodowany ciąg symboli jest dzielony na bloki $k$ symboli i każdy jest niezależnie zastępowany odpowiednim blokiem $n$ symboli.

W związku z tym, że słowa kodowe mogą reprezentować wszystkie możliwe kombinacje symboli w ciągu informacyjnym, to różnych słów w kodzie jest $2^k$. Ale wszystkich ciągów o długości $n$ jest $2^n$, zatem przeważająca część ciągów o długości $n$ reprezentuje sytuację, w której nastąpiło przekłamanie (są to tzw. sekwencje zabronione).

## Macierze opisujące systematyczny kod liniowy

### Macierz generująca

Macierz generująca zawiera bazę podprzestrzeni liniowej (tj. naszego kodu). Można jej używać do tworzenia słów kodowych $\mathbf{x} $ z ciągów informacyjnych $\mathbf{u}$:

$$
  \mathbf{x} = \mathbf{u}\mathbf{G}
$$

Ciąg informacyjny liczy $k$ symboli, słowo kodowe $n$ symboli, zatem macierz generująca ma wymiary $k \times n$.

Na przykład dla kodu $(5,3)$ możemy mieć następującą macierz generującą:

$$
  \mathbf{G} = \begin{bmatrix} 
    1 & 0 & 0 & 1 & 0 \\
    0 & 1 & 0 & 1 & 1 \\
    0 & 0 & 1 & 0 & 1
  \end{bmatrix}
$$

W tym przypadku możemy zobaczyć, w jaki sposób symbole ciągu informacyjnego odwzorowują się na symbole słowa kodowego:

$$
  \begin{bmatrix} x_1 & x_2 & x_3 & x_4 & x_5 \end{bmatrix} = \begin{bmatrix} u_1 & u_2 & u_3 \end{bmatrix}\begin{bmatrix} 
    1 & 0 & 0 & 1 & 0 \\
    0 & 1 & 0 & 1 & 1 \\
    0 & 0 & 1 & 0 & 1
  \end{bmatrix}
$$

Zatem (musimy pamiętać, że wykonujemy operacje w $\mathit{GF}(2)$):
$$
  x_1 = u_1
$$

$$
  x_2 = u_2
$$

$$
  x_3 = u_3
$$

$$
  x_4 = u_1 \oplus u_2 = x_1 \oplus x_2
$$

$$
  x_5 = u_2 \oplus u_3 = x_2 \oplus x_3
$$

Dla kodu systematycznego $(n,k)$ macierz generująca może być podana w postaci kanonicznej i występuje w niej na początku podmacierz jednostkowa $\mathbf{I}_k$.

In [ ]:
#@markdown Prosty kod liniowy: macierz generująca

G = np.array([
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 1],
    [0, 0, 1, 0, 1]
])

n = G.shape[1]
k = G.shape[0]

print(f"Macierz generująca dla naszego kodu ({n},{k}):\n{G}")

kod = np.zeros((2**k, n), dtype=int)
d_min = n
print(f"\nWszystkie słowa kodowe: ")
for i in range(2**k):
  u = np.array([int(x) for x in np.binary_repr(i, width=k)])
  x = np.matmul(u,G) % 2
  waga_x = np.sum(x)
  print(f"-> dla ciągu {u}: słowo kodowe {x} o wadze {waga_x}.")
  if waga_x > 0 and waga_x < d_min:
    d_min = waga_x
  kod[i] = x

print("\nCały kod w jednej macierzy:\n", kod)

Macierz generująca dla naszego kodu (5,3):
[[1 0 0 1 0]
 [0 1 0 1 1]
 [0 0 1 0 1]]

Wszystkie słowa kodowe: 
-> dla ciągu [0 0 0]: słowo kodowe [0 0 0 0 0] o wadze 0.
-> dla ciągu [0 0 1]: słowo kodowe [0 0 1 0 1] o wadze 2.
-> dla ciągu [0 1 0]: słowo kodowe [0 1 0 1 1] o wadze 3.
-> dla ciągu [0 1 1]: słowo kodowe [0 1 1 1 0] o wadze 3.
-> dla ciągu [1 0 0]: słowo kodowe [1 0 0 1 0] o wadze 2.
-> dla ciągu [1 0 1]: słowo kodowe [1 0 1 1 1] o wadze 4.
-> dla ciągu [1 1 0]: słowo kodowe [1 1 0 0 1] o wadze 3.
-> dla ciągu [1 1 1]: słowo kodowe [1 1 1 0 0] o wadze 3.

Cały kod w jednej macierzy:
 [[0 0 0 0 0]
 [0 0 1 0 1]
 [0 1 0 1 1]
 [0 1 1 1 0]
 [1 0 0 1 0]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [1 1 1 0 0]]


In [ ]:
#@markdown Minimalna odległość Hamminga $d_{\min}$ dla kodu liniowego

def hamming_distance(x: np.ndarray, y: np.ndarray) -> int:
  roznice = np.array(x != y)
  return np.count_nonzero(roznice)

print("Porównanie odległości Hamminga między wszystkimi parami różnych słów kodowych: ")

for i in range(kod.shape[0]):
  for j in range(i+1,kod.shape[0]):
    print(f"-> odległość między {kod[i]} i {kod[j]} wynosi {hamming_distance(kod[i], kod[j])}.")

print(f"\nMinimalna odległość Hamminga dla tego kodu (wyliczona z niezerowych wag słów kodowych): {d_min}.")

Porównanie odległości Hamminga między wszystkimi parami różnych słów kodowych: 
-> odległość między [0 0 0 0 0] i [0 0 1 0 1] wynosi 2.
-> odległość między [0 0 0 0 0] i [0 1 0 1 1] wynosi 3.
-> odległość między [0 0 0 0 0] i [0 1 1 1 0] wynosi 3.
-> odległość między [0 0 0 0 0] i [1 0 0 1 0] wynosi 2.
-> odległość między [0 0 0 0 0] i [1 0 1 1 1] wynosi 4.
-> odległość między [0 0 0 0 0] i [1 1 0 0 1] wynosi 3.
-> odległość między [0 0 0 0 0] i [1 1 1 0 0] wynosi 3.
-> odległość między [0 0 1 0 1] i [0 1 0 1 1] wynosi 3.
-> odległość między [0 0 1 0 1] i [0 1 1 1 0] wynosi 3.
-> odległość między [0 0 1 0 1] i [1 0 0 1 0] wynosi 4.
-> odległość między [0 0 1 0 1] i [1 0 1 1 1] wynosi 2.
-> odległość między [0 0 1 0 1] i [1 1 0 0 1] wynosi 3.
-> odległość między [0 0 1 0 1] i [1 1 1 0 0] wynosi 3.
-> odległość między [0 1 0 1 1] i [0 1 1 1 0] wynosi 2.
-> odległość między [0 1 0 1 1] i [1 0 0 1 0] wynosi 3.
-> odległość między [0 1 0 1 1] i [1 0 1 1 1] wynosi 3.
-> odległość między [0 1

### Macierz kontrolna (macierz kontroli parzystości)

Słowa kodowe tworzymy w taki sposób, że możemy opisać pojedyncze słowo, w którym da się wyróżnić symbole / pozycje informacyjne i kontrolne (tzn. na pewno da się to zrobić dla kodu systematycznego, dla kodu niesystematycznego może nie dać się w taki sposób wyróżnić pozycji).

Na przykład jeśli mamy słowo (trzymamy się tylko symboli binarnych):
$$ x_1 x_2 x_3 x_4 x_5 $$
to możemy mieć kod, w przypadku którego pierwsze trzy symbole są informacyne, a dwa pozostałe tworzone jako kontrolne z tych trzech pierwszych, np.:
$$ x_4 = x_1 \oplus x_2 \Longrightarrow x_1 \oplus x_2 \oplus x_4 = 0 $$
$$ x_5 = x_2 \oplus x_3 \Longrightarrow x_2 \oplus x_3 \oplus x_5 = 0 $$

W każdym takim przypadku równania kontrolne są opisane tzw. macierzą kontrolną (ona nazywana jest też macierzą kontroli parzystości, ale to nie musi być najprostsza kontrola parzysości, którą znamy z lat wcześniejszych), która w tym przypadku wygląda następująco:

$$  
  \mathbf{H} = \begin{bmatrix} 
    1 & 1 & 0 & 1 & 0 \\
    0 & 1 & 1 & 0 & 1 \\
  \end{bmatrix}
$$

Wymiary też macierzy to $(n-k) \times n$, czyli wierszy jest tyle, ile symboli kontrolnych, a kolumn tyle ile symboli w pojedynczym słowie.

Skoro macierz kontrolna zawiera równania kontrolne, które są spełnione dla słowa kodowego, to musi zachodzić:
$$
  \mathbf{x}\mathbf{H}^T = \mathbf{0}_{n-k}
$$
w przeciwnym przypadku zachodzi przekłamanie.

Wynik mnożenia $\mathbf{s} = \mathbf{x}\mathbf{H}^T$ nazywa się syndromem.

Każde słowo w kodzie liniowym jest kombinacją wektorów bazy zawartych w macierzy generującej, musi zatem zachodzić:
$$
  \mathbf{G}\mathbf{H}^T = \mathbf{0}_{k\times(n-k)}
$$

Dla kodu systematycznego $(n,k)$ macierz kontrolna może być podana w postaci kanonicznej i występuje w niej na końcu podmacierz jednostkowa $\mathbf{I}_{n-k}$. W takim przypadku macierz kontrolna może być bardzo prosto uzyskana z generującej (i na odwrót).

In [ ]:
#@markdown Prosty kod liniowy: macierz kontrolna

def H_z_G(Gen: np.ndarray) -> np.ndarray:
  k = Gen.shape[0]
  if not np.allclose(np.eye(k), Gen[:k,:k]):
    print("G nie jest w postaci kanonicznej!")
    return
  PT = Gen[:,k:]
  H = np.concatenate((PT.T,np.eye(n-k)), axis=1).astype(int)
  print(f"Dla macierzy generującej G = \n{Gen}\npodmacierz P^T = \n{PT}\nzatem macierz kontrolna H = \n{H}")
  return H

H = H_z_G(G)

print("*"*100)
print(f"GH^T = \n{np.matmul(G,H.T) % 2}")
print("*"*100)
print("Cały kod w jednej macierzy:\n", kod)
print("*"*100)
print(f"Wyniki mnożenia yH^T dla wszystkich y z V_n:")
for i in range(2**H.shape[1]):
  y = np.array([int(x) for x in np.binary_repr(i, width=H.shape[1])])
  s = np.matmul(y,H.T) % 2
  print(f"-> dla ciągu {y} syndrom s = {s}, zatem", end=" ")
  if np.sum(s) == 0:
    print("jest to poprawne słowo kodowe")
  else:
    print("jest to sekwencja zabroniona")

Dla macierzy generującej G = 
[[1 0 0 1 0]
 [0 1 0 1 1]
 [0 0 1 0 1]]
podmacierz P^T = 
[[1 0]
 [1 1]
 [0 1]]
zatem macierz kontrolna H = 
[[1 1 0 1 0]
 [0 1 1 0 1]]
****************************************************************************************************
GH^T = 
[[0 0]
 [0 0]
 [0 0]]
****************************************************************************************************
Cały kod w jednej macierzy:
 [[0 0 0 0 0]
 [0 0 1 0 1]
 [0 1 0 1 1]
 [0 1 1 1 0]
 [1 0 0 1 0]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [1 1 1 0 0]]
****************************************************************************************************
Wyniki mnożenia yH^T dla wszystkich y z V_n:
-> dla ciągu [0 0 0 0 0] syndrom s = [0 0], zatem jest to poprawne słowo kodowe
-> dla ciągu [0 0 0 0 1] syndrom s = [0 1], zatem jest to sekwencja zabroniona
-> dla ciągu [0 0 0 1 0] syndrom s = [1 0], zatem jest to sekwencja zabroniona
-> dla ciągu [0 0 0 1 1] syndrom s = [1 1], zatem jest to sekwencja zabroniona
-> dl

# Kod z kontrolą parzystości jako kod liniowy

Zapewne najbardziej popularnym dydaktycznie :) kodem detekcyjnym jest kod z kontrolą parzystości. Ma parametry:

$$ (n, n-1) \quad \text{tj.} \quad n = k+1$$

Kodowanie jest bardzo proste: do $n-1 = k$ symboli tworzących ciąg informacyjny dodaje się jeden symbol kontrolny w taki sposób, żeby w całym ciągu $n$ symboli była parzysta liczba jedynek.

W ten sposób: jeśli wystąpi jedno przekłamanie na dowolnym z $n$ symboli w bloku, będzie wiadomo, że nastąpiło przekłamanie, bo liczba jedynek będzie nieparzysta.

Jak automatycznie wyliczać bit kontrolny? Zobaczmy, że bit kontrolny ma być 0, jeśli tylko symbole informacyjne zawierają parzystą liczbę 1, a takie coś "wykrywa" dodawanie XOR:

$$x_1 \oplus x_2 \oplus x_3 \oplus \dots \oplus x_{n-1} = x_n$$

a więc:

$$x_1 \oplus x_2 \oplus x_3 \oplus \dots \oplus x_{n-1} \oplus x_n = 0$$

czyli:

$$\mathbf{H} = \begin{bmatrix} 1 & 1 & 1 & \cdots & 1 & | 1 \end{bmatrix}$$

In [ ]:
#@title Syndrom dla kodu z kontrolą parzystości { run: "auto" }

#@markdown Wprowadź ciąg binarny:
slowo = '00111100' #@param {type:"string"}

y = np.array([int(c) for c in slowo])
H = np.ones(len(y), dtype=int)

print(f"Macierz kontrolna: {H}")

# syndrom = int(H @ y % 2)
syndrom = int(np.dot(H,y)%2)

print(f"Syndrom to {syndrom}, zatem: ", end="")
if syndrom:
  print("na pewno nastąpiły przekłamania.")
else:
  print("możemy wnioskować, że nie ma przekłamań (chociaż możemy się mylić).")

Macierz kontrolna: [1 1 1 1 1 1 1 1]
Syndrom to 0, zatem: możemy wnioskować, że nie ma przekłamań (chociaż możemy się mylić).


## Macierz generująca dla kodu z kontrolą parzystości

Skoro:
$$
  \mathbf{H} = \begin{bmatrix} 1 & 1 & 1 & \cdots & 1 & | 1 \end{bmatrix} = \mathbf{H} = \begin{bmatrix} \mathbf{P} | 1 \end{bmatrix}
$$

to macierz generująca:
$$
  \mathbf{G} = \begin{bmatrix} \mathbf{I}_{n-1} | \mathbf{P}^T \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 & \cdots & 0 & 0 & 1 \\ 0 & 1 & 0 & \cdots & 0 & 0 & 1 \\ 0 & 0 & 1 & \cdots & 0 & 0 & 1 \\  &  &  &  & \ddots & & \\ 0 & 0 & 0 & \cdots & 1 & 0 & 1 \\ 0 & 0 & 0 & \cdots & 0 & 1 & 1 \end{bmatrix}
$$

Jest tak dlatego, że dla ciągu informacyjnego $\mathbf{u}$ tworzymy słowo kodowe:
$$
  \mathbf{x} = \mathbf{u}\mathbf{G}
$$
według reguły:
$$
  \forall_{i=1,\dots,n-1} \quad x_i = u_i \\
  x_n = \sum_{i=1}^{n-1} u_i = \sum_{i=1}^{n-1} x_i
$$


In [ ]:
#@markdown Macierz generująca kodu z kontrolą parzystości

def G_z_H(Kon: np.ndarray) -> np.ndarray:
  n = Kon.shape[1]
  k = n - Kon.shape[0]
  print(f"Kod ({n},{k})")

  if not np.allclose(np.eye(n-k), Kon[:,k:]):
    print("H nie jest w postaci kanonicznej!")
    return
  P = Kon[:,:k]
  G = np.concatenate((np.eye(k),P.T), axis=1).astype(int)
  print(f"Dla macierzy kontrolnej H = \n{Kon}\npodmacierz P = \n{P}\nzatem macierz generująca G = \n{G}")
  return G, k

K = np.array([H])
G, k = G_z_H(K)

print("*"*100)
print("Sprawdźmy, czy poprawnie koduje: ")
for i in range(2**k):
  u = np.array([int(x) for x in np.binary_repr(i, width=k)])
  x = np.matmul(u,G) % 2
  s = np.matmul(x,K.T) % 2
  print(f"-> ciąg informacyjny {u} słowo kodowe x = {x}; wynik mnożenia xH^T = {s}.")

Kod (8,7)
Dla macierzy kontrolnej H = 
[[1 1 1 1 1 1 1 1]]
podmacierz P = 
[[1 1 1 1 1 1 1]]
zatem macierz generująca G = 
[[1 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 1]
 [0 0 0 1 0 0 0 1]
 [0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 1]]
****************************************************************************************************
Sprawdźmy, czy poprawnie koduje: 
-> ciąg informacyjny [0 0 0 0 0 0 0] słowo kodowe x = [0 0 0 0 0 0 0 0]; wynik mnożenia xH^T = [0].
-> ciąg informacyjny [0 0 0 0 0 0 1] słowo kodowe x = [0 0 0 0 0 0 1 1]; wynik mnożenia xH^T = [0].
-> ciąg informacyjny [0 0 0 0 0 1 0] słowo kodowe x = [0 0 0 0 0 1 0 1]; wynik mnożenia xH^T = [0].
-> ciąg informacyjny [0 0 0 0 0 1 1] słowo kodowe x = [0 0 0 0 0 1 1 0]; wynik mnożenia xH^T = [0].
-> ciąg informacyjny [0 0 0 0 1 0 0] słowo kodowe x = [0 0 0 0 1 0 0 1]; wynik mnożenia xH^T = [0].
-> ciąg informacyjny [0 0 0 0 1 0 1] słowo kodowe x = [0 0 0 0 1 0 1 0]; wynik mnożenia xH^T = [0].
-> ciąg 

# Kod z powtarzaniem jako kod liniowy

Najbardziej prymitywny kod korekcyjny to kod z powtarzaniem. Polega na $(n-1)$-krotnym powtórzeniu symbolu, tzn. na jeden bit informacyjny przypada $n-1$ identycznych z nim bitów kontrolnych (innymi słowy: na jedną pozycję informacyjną / jeden symbol informacyjny przypada $n-1$ pozycji / symboli kontrolnych):

$$ x_1 = x_2 $$
$$ x_1 = x_3 $$
$$ x_1 = x_4 $$
$$ \vdots $$
$$ x_1 = x_n $$

czyli:

$$ x_1 \oplus x_2 = 0 $$
$$ x_1 \oplus x_3 = 0 $$
$$ x_1 \oplus x_4 = 0 $$
$$ \vdots $$
$$ x_1 \oplus x_n = 0 $$

a więc macierz kontrolna dla takiego kodu o parametrach $(n,1)$ przyjmuje formę:

$$\mathbf{H} = \begin{bmatrix} 1 & 1 & 0 & 0 & \cdots & 0 & 0 \\ 1 & 0 & 1 & 0 & \cdots & 0 & 0 \\ 1 & 0 & 0 & 1 & \cdots & 0 & 0 \\  &  &  &  & \ddots & & \\ 1 & 0 & 0 & 0 & \cdots & 1 & 0 \\ 1 & 0 & 0 & 0 & \cdots & 0 & 1 \end{bmatrix}$$

Kod z powtarzaniem może oczywiście polegać na wielokrotnym powtarzaniu całego bloku bitów, np. dla trzech powtórzeń:

$$111 \rightarrow 111\,111\,111\,111$$
$$0101 \rightarrow 0101\,0101\,0101\,0101$$

jednakże w takim przypadku nie da się go w jednolity sposób przedstawić jako kodu $(n,k)$.

Kod z powtarzaniem o parametrach $(n,1)$ pozwala nam bezbłędnie korygować wszystkie przekłamania aż do $\lfloor \frac{n}{2} \rfloor$-krotnych.

In [ ]:
#@title  { run: "auto" }
#@markdown Macierz generująca kodu z powtarzaniem

#@markdown Wprowadź $n$:
n = 5 #@param {type:"integer"}

assert n>1, "to ma być kod z powtarzaniem"

H = np.concatenate((np.ones((n-1,1)),np.eye(n-1)), axis=1).astype(int)

G, k = G_z_H(H)

print("*"*100)
print("Sprawdźmy, czy poprawnie koduje: ")
for i in range(2**k):
  u = np.array([int(x) for x in np.binary_repr(i, width=k)])
  x = np.matmul(u,G) % 2
  s = np.matmul(x,H.T) % 2
  print(f"-> ciąg informacyjny {u} słowo kodowe x = {x}; wynik mnożenia xH^T = {s}.")

Kod (5,1)
Dla macierzy kontrolnej H = 
[[1 1 0 0 0]
 [1 0 1 0 0]
 [1 0 0 1 0]
 [1 0 0 0 1]]
podmacierz P = 
[[1]
 [1]
 [1]
 [1]]
zatem macierz generująca G = 
[[1 1 1 1 1]]
****************************************************************************************************
Sprawdźmy, czy poprawnie koduje: 
-> ciąg informacyjny [0] słowo kodowe x = [0 0 0 0 0]; wynik mnożenia xH^T = [0 0 0 0].
-> ciąg informacyjny [1] słowo kodowe x = [1 1 1 1 1]; wynik mnożenia xH^T = [0 0 0 0].


# Kod Hamminga: kod liniowy szczelnie upakowany

Kod Hamminga charakteryzuje się macierzą kontrolną, którą tworzą reprezentacje binarne kolejnych liczb. Parametry kodu to dla dowolnego $m \geq 3$: $(2^m-1,2^m-1-m)$. Zatem długość słowa kodowego $n= 2^m-1$, liczba symboli informacyjnych to $k = 2^m-1-m$, zaś symboli kontrolnych jest w tym przypadku $n-k = m$. Najprostszy taki kod ma parametry $(7,4)$ i macierz kontrolną:

$$  
  \mathbf{H} = \begin{bmatrix} 
    0 & 0 & 0 & 1 & 1 & 1 & 1 \\                              
    0 & 1 & 1 & 0 & 0 & 1 & 1 \\
    1 & 0 & 1 & 0 & 1 & 0 & 1
  \end{bmatrix} 
  \Longrightarrow
  \begin{cases}
    x_4 \oplus x_5 \oplus x_6 \oplus x_7 = 0 \\
    x_2 \oplus x_3 \oplus x_6 \oplus x_7 = 0 \\
    x_1 \oplus x_3 \oplus x_5 \oplus x_7 = 0
  \end{cases}
  \Longrightarrow
  \begin{cases}
    x_4 = x_5 \oplus x_6 \oplus x_7 \\
    x_2 = x_3 \oplus x_6 \oplus x_7 \\
    x_1 = x_3 \oplus x_5 \oplus x_7
  \end{cases}
$$

Mnemotechnicznie - patrząc od prawej strony od dołu:
*   w ostatnim rzędzie mamy naprzemiennie jedynkę z zerem,
*   w następnym rzędzie mamy naprzemiennie dwie jedynki z dwoma zerami,
*   w następnym rzędzie mamy naprzemiennie cztery jedynki...
W ogólności w rzędzie nr $i$ mamy naprzemiennie $2^{i-1}$ jedynek z $2^{i-1}$ zerami (na końcu zer jest o jedno mniej).

Proszę zauważyć, że kolejne kolumny macierzy Hamminga tworzą binarne reprezentacje liczb: $1, 2, 3, \dots, 2^n-1$. Ze względu na powyższą właściwość korekta jest prosta: wartość syndromu (w postaci liczby binarnej) mówi, którą pozycję skorygować.

Korygować można tylko jedną pozycję ($t=1$), ponieważ minimalna odległość Hamminga dla kodu Hamminga wynosi $d_{\min} = 1$ (można to sprawdzić na macierzy kontrolnej: wszystkie kolumny są różne, a da się dobrać trzy różne kolumny sumujące się do $\begin{bmatrix} 0 \\ 0 \\ 0 \end{bmatrix}$).

W przypadku kodu Hamminga pozycje informacyjne oraz kontrolne są wymieszane (jest to tzw. kod niesystematyczny), ale i tak można poznać, które pozycje są kontrolne: to pozycji związane z kolumnami, które zawierają tylko jedną jedynkę (czyli pozycje, które występują w jednym równaniu kontrolnym, które je wyznacza).

Najpierw znajdźmy wszystkie słowa kodowe kodu Hamminga:

In [ ]:
#@title Sprawdźmy, czy konkretne słowo należy do kodu Hamminga $(7,4)$ { run: "auto" }
#@markdown * Przy okazji od razu zobaczymy, jak działa dekodowanie z syndromem
#@markdown * Polecam sprawdzić słowo 1010101 i jego małe modyfikacje

def dekodowanie_syndrom(H, y):
  syndrom = np.matmul(y,H.T) % 2
  print(f"Podano słowo {y}, syndrom dla tego słowa to {syndrom}, zatem:")
  if sum(syndrom) == 0:
    print("uznajemy, że nie nastąpiło przekłamanie")
  else:
    brak = 8 - len(syndrom)
    if brak > 0:
      padd = np.zeros(brak, int)
      syndrom = np.hstack((padd,syndrom))
    pozycja = int(np.packbits(syndrom))
    y[pozycja-1] += 1
    poprawione = y%2
    print(f"uznajemy, że nastąpiło przekłamanie na pozycji {pozycja} i poprawne słowo kodowe to {poprawione}")

#@markdown Wprowadź ciąg binarny:
slowo = '1010101' #@param {type:"string"}

assert len(slowo) == 7, "Oczekiwane słowo o długości 7"

H = np.array(
    [
        [0, 0, 0, 1, 1, 1, 1],
        [0, 1, 1, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0, 1]
    ]
)

print(f"Macierz kontrolna kodu Hamminga (7,4):\n{H}")

y = np.array([int(x) for x in slowo])
dekodowanie_syndrom(H, y) 

Macierz kontrolna kodu Hamminga (7,4):
[[0 0 0 1 1 1 1]
 [0 1 1 0 0 1 1]
 [1 0 1 0 1 0 1]]
Podano słowo [1 0 1 0 1 0 1], syndrom dla tego słowa to [0 0 0], zatem:
uznajemy, że nie nastąpiło przekłamanie


Warto się pobawić z kodami Hamminga wyższych rzędów...